In [1]:
import OMPython
from OMPython import OMCSessionZMQ

In [2]:
omc = OMCSessionZMQ()

In [3]:
omc.sendExpression("getVersion()")

'OpenModelica v1.16.2 (64-bit)'

In [4]:
omc.sendExpression("cd()")

'C:/Users/uapug'

In [5]:
omc.sendExpression("loadModel(Modelica)")

True

In [6]:
omc.sendExpression("loadFile(getInstallationDirectoryPath() + \"/share/doc/omc/testmodels/Disinfection.mo\")")

True

In [7]:
omc.sendExpression("instantiateModel(Disinfection)")

'class Disinfection\n  Real N(start = 1000.0);\n  Real C(start = 0.3) "mg/L";\n  constant Real lamda_CW = 0.0017 " s*mg/L)^-1";\n  constant Real k_growth = 3.3e-005 "1/s";\n  constant Real k_decay = 0.00017 "1/s";\ninitial equation\n  N = 1000.0;\n  C = 0.3;\nequation\n  der(C) = (-0.00017) * C;\n  if C >= 0.01 then\n    der(N) = (-0.0017) * C * N;\n  else\n    der(N) = 3.3e-005 * N;\n  end if;\nend Disinfection;\n'

In [8]:
cmds = [
  'loadFile(getInstallationDirectoryPath() + "/share/doc/omc/testmodels/Disinfection.mo")',
  "simulate(Disinfection)",
  "plot(der(C))"
  ]

cmds

In [9]:
cmds

['loadFile(getInstallationDirectoryPath() + "/share/doc/omc/testmodels/Disinfection.mo")',
 'simulate(Disinfection)',
 'plot(der(C))']

In [10]:
for cmd in cmds:
    answer = omc.sendExpression(cmd)
    print("\n{}:\n{}".format(cmd, answer))


loadFile(getInstallationDirectoryPath() + "/share/doc/omc/testmodels/Disinfection.mo"):
True

simulate(Disinfection):
{'resultFile': 'C:/Users/uapug/Disinfection_res.mat', 'simulationOptions': "startTime = 0.0, stopTime = 1.0, numberOfIntervals = 500, tolerance = 1e-006, method = 'dassl', fileNamePrefix = 'Disinfection', options = '', outputFormat = 'mat', variableFilter = '.*', cflags = '', simflags = ''", 'messages': 'LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n', 'timeFrontend': 0.5812588, 'timeBackend': 0.0307314, 'timeSimCode': 0.0077575, 'timeTemplates': 0.1028909, 'timeCompile': 20.023261, 'timeSimulation': 0.4689131, 'timeTotal': 21.2181127}

plot(der(C)):
True
